Here is the final scraper that we will be using to populate the database.  We will be pdf scraping since there were technical limitations scraping the course website. (likely protections from UCLA) 

For the demo purposes, we will have around 10 majors with 10 classes each, with 4 discussion sections each, totalling 400 unique class codes.  More information about why we chose this amount of data is outlined below and in previous web scraping attempts.

In [105]:
#importing necessary packages
import PyPDF2
import re

In [106]:
#putting part of the pdf of the UCLA catalog into a string
#takes a while to run, ~1 minute depending on device
def extract_text_from_pdf(pdf_path, start_page=180, end_page=890):
    text = ""
    #opens pdf
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        start_page = max(0, start_page - 1)  
        end_page = min(end_page, len(reader.pages))
        #chooses which pages to parse through - the class section of the pdf in this case
        for page_number in range(start_page, end_page):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text


#use path to course catalog pdf on your device
class_text = extract_text_from_pdf('/Users/dayfamily/Desktop/UCLA_Catalog_2023-24.pdf', start_page=180, end_page=890)


How we are going to input what we scrape into the database:

INSERT INTO classes VALUES ('course_code', 'dept_code', 'course_num', 'course_name', 'prof', 'discussion_code');

ex:  INSERT INTO classes VALUES ('CS000331A', 'COM SCI', '33', 'Introduction to Computer Organization', 'Reinman, G.D.', '1A');

So, we will need to make an array that has the following elements: course_code, dept_code, course_num, course_name, prof, and discussion_code.  Unfortunately, due to the inability to scrape the registrar, we will have to use dummy variables for prof and discussion_code, as those things will be variate by quarter and not something we can confirm off of the pdf version of the course catalog.  Additionally, assigning dept_code will be a little bit tricky.    

First we will extract course names and course numbers from the pdf.

In [107]:
#a regular expression that looks for a specific pattern that matches how the pdf stores info about course_names and course_codes; not completely perfect, especially when it comes to things like fiat lux or research
course_pattern = r'(\b[A-Za-z]*\d+[A-Za-z]?)\.\s(.+?)\.'
course_matches = re.findall(course_pattern, class_text)



In [ ]:
#check that it works
print(course_matches[:5]) 

In [108]:
#preliminary array using what info we got so far
courses_array = []
for match in course_matches:
    course_code = match[0]
    course_name = match[1]
    courses_array.append(["", "", course_code, course_name, "", ""])


In [ ]:
#check that it works
for row_index, row_data in enumerate(courses_array[:12000], start=1):
    print(f"Row {row_index}: {row_data}")

There are over 10,000 classes (with ~4 sections each), and there is no easy way to assign a major to each of them using the information we have.  I attempted to use different regular expressions to scrape major names on a given page and then I could subsequently assign the classes on that page to that major, but to no avail with the given formatting.  Thus, the only way to assign majors to each course, would be to do so manually, and I will not be doing all 10,000+ courses.  Instead, I can do it "semi-manually", where I look at which range each class is, and assign the dept_code to those courses.  I could also write a program to do that based on perhaps if the next course number is lower than the previous one, and to change the dept_code accordingly, but that is prone to errors and having incorrect information seems worse than having incomplete information.  Additionally, even if I did that, we wouldn't be able to populate our database with all of that information so it seems like not the best use of our time.  So, I will input dept_code information "Semi-manually" for the time being.  I will do the first couple of majors, and then prioritize popular majors like computer science, mathematics, and political science.  

In [109]:
#complete list of dept_codes, found on https://github.com/nnhien/uclacatalog/wiki
dept_abbr = { "POPULATED FROM LINK ABOVE; REMOVED FROM THIS NB FOR BREVITY" }

This is the beginning of "Semi-manually" assigning majors to classes - I will do the following majors: AERO ST, AF AMER, POLI SCI, MATH, COM SCI, ECON, PHILOS, SOC SC, THEATER, EC ENGR.  

In [110]:
#I will essentially be copying this cell for as many majors as I fill out - replacing the dept_code and range accordingly.  I will be determining the range by looking at the last class listed for each major on the pdf of the general catalog.

#here is the first one: AERO ST

# Given dept_code to assign
dept_code = "AERO ST"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(0,5):
    courses_array[row_index][1] = dept_code

In [111]:

# Given dept_code to assign
dept_code = "EC ENGR"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(3815,3830):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[3815:3830], start=1):
    print(f"Row {row_index}: {row_data}")

In [112]:

# Given dept_code to assign
dept_code = "THEATER"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(10521,10537):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[10520:10538], start=1):
    print(f"Row {row_index}: {row_data}")

In [113]:

# Given dept_code to assign
dept_code = "PUB AFF"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(9473,9488):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[9473:9488], start=1):
    print(f"Row {row_index}: {row_data}")

In [114]:

# Given dept_code to assign
dept_code = "PHILOS"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(8568,8585):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[8568:8586], start=1):
    print(f"Row {row_index}: {row_data}")

In [115]:

# Given dept_code to assign
dept_code = "ECON"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(3397,3414):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[3397:3415], start=1):
    print(f"Row {row_index}: {row_data}")

In [116]:
# Given dept_code to assign
dept_code = "AF AMER"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(5,102):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the relevant section of the updated array
for row_index, row_data in enumerate(courses_array[4:14], start=1):
    print(f"Row {row_index}: {row_data}")

In [117]:
# Given dept_code to assign
dept_code = "POLI SCI"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(8997,9010):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the relevant section of the updated array
for row_index, row_data in enumerate(courses_array[8996:9011], start=1):
    print(f"Row {row_index}: {row_data}")

In [118]:
# Given dept_code to assign
dept_code = "COM SCI"

# Assigning the dept_code to the second column of however many classes it corresponds to (LIMITING TO AROUND 10 FOR DB PURPOSES)
for row_index in range(2793,2810):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the relevant section of the updated array
for row_index, row_data in enumerate(courses_array[2792:2811], start=1):
    print(f"Row {row_index}: {row_data}")

In [119]:
# Given dept_code to assign
dept_code = "MATH"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(6975,7000):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the relevant section of the updated array
for row_index, row_data in enumerate(courses_array[6974:7001], start=1):
    print(f"Row {row_index}: {row_data}")

Okay, using the classes that I have filled out a dept_code for, I will now make a new array and fill out dummy discussion sections for them.

In [120]:
# Create a new array containing only rows with non-empty second elements
discussions_template_array = [row for row in courses_array if row[1] != ""]



In [144]:
for row in discussions_template_array:
    row[5] = "1A"



In [ ]:
for row in discussions_template_array:
    print(row)

In [149]:
discussions_updated_array = []

for row in discussions_template_array:
    # Create a duplicate row with the value of the sixth column changed to "1B"
    duplicate_row_1b = row[:]  # Create a shallow copy of the original row
    duplicate_row_1b[5] = "1B"  # Update the value of the sixth column to "1B"
    duplicate_row_1c = row[:]  # Create a shallow copy of the original row
    duplicate_row_1c[5] = "1C"  # Update the value of the sixth column to "1B"
    duplicate_row_1d = row[:]  # Create a shallow copy of the original row
    duplicate_row_1d[5] = "1D"  # Update the value of the sixth column to "1B"
    
    # Append the original row and the duplicate row to the updated array
    discussions_updated_array.append(row)
    discussions_updated_array.append(duplicate_row_1b)
    discussions_updated_array.append(duplicate_row_1c)
    discussions_updated_array.append(duplicate_row_1d)



In [ ]:
for row in discussions_updated_array:
    print(row)

Now that each class has 4 discussion sections, a dept code, a course code, and a course name, I will assign a dummy variable to be their professor - I will update it with more apt names when adding it to the website.  

You can create a list of professors from the UCLA registrar - no scraping required.

In [123]:
dummy_profs = dummy_profs.split('\n')

In [124]:

dummy_profs = dummy_profs[1:-1]

In [ ]:
print(dummy_profs[1:-1])

In [125]:
import random

for row in discussions_template_array:
    random_index = random.randint(0, len(dummy_profs) - 1)
    row[4] = dummy_profs[random_index]


In [ ]:
# Print the modified discussions_updated_array
for row in discussions_updated_array:
    print(row)

Now, all I need to have some classes completely filled out and ready to populate the database is to give each class a unique course_code.  This will look like the following:

2 letters representing the dept_code i.e. CS for COMP SCI
5 characters representing the course_num, i.e. 00001 for Introduction to Black Studies
2 characters representing the discussion_code i.e. 1C


In [151]:
for row in discussions_updated_array:
    # Extract relevant information from the row
    dept_code = row[1]
    course_num = row[2]
    discussion_code = row[5]
    
    # Construct the course code
    if dept_code == 'COM SCI':
        course_code = 'CS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'AERO ST':
        course_code = 'AS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'POLI SCI':
        course_code = 'PS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'MATH':
        course_code = 'MT' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'PHILOS':
        course_code = 'PH' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'PUB AFF':
        course_code = 'PA' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'THEATER':
        course_code = 'TH' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'EC ENGR':
        course_code = 'EE' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'ECON':
        course_code = 'EC' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'AF AMER':
        course_code = 'AA' + '0' * (5 - len(course_num)) + course_num + discussion_code
        pass

    
    # Update the first column of the row with the constructed course code
    row[0] = course_code


In [152]:
#rename for clarity
final_courses_array = discussions_updated_array

In [ ]:
for row_index, row_data in enumerate(final_courses_array, start=1):
    print(f"Row {row_index}: {row_data}")

Here I will attempt to insert a few classes into our database before I populate it completely to avoid crashing the server

In [222]:
test_courses_array = final_courses_array[908:958]

In [ ]:
test_courses_array

The next cell is the cell that actually populates the database!!

In [223]:

import psycopg2
#replace the user, password, etc with actual data
connection = psycopg2.connect(user="USER", password="PASSWRD", host="HOST", port="PORT", database="DB")
connection.autocommit = True
with connection.cursor() as cur: 
    for row in test_courses_array:
        course_code, dept_code, course_num, course_name, prof, discussion_code = row
        cur.execute (f"""
        INSERT INTO classes 
            VALUES ('{course_code}', '{dept_code}', '{course_num}', '{course_name}', '{prof}', '{discussion_code}') 
            ON CONFLICT (section_code) DO NOTHING;
        """)
connection.close()



below are failed attempts to form regular expressions that capture majors so that I can associate them with courses. 

In [ ]:
unique_first_column_values = {}  
unique_rows = []  


for row in discussions_updated_array:
  
    if row[0] not in unique_first_column_values:
       
        unique_first_column_values[row[0]] = True
        unique_rows.append(row)


final_courses_array = unique_rows